In [1]:
import sys
sys.path.insert(0, '../')

import pandas as pd
from preprocess import Audio_Processor
from sklearn import metrics
import matplotlib.pyplot as plt
from data_utils import enumerate_strings

Using TensorFlow backend.


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
SR = 16000
blocksize = int(SR/2)
overlap = int(SR/4)

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Setup

In [4]:
path_to_db='../../ESC-50/'
ps = Audio_Processor(path_to_db + 'audio/', sr=SR)
dataset = pd.read_csv(path_to_db + 'meta/esc50.csv')
classes = [None] * 50
h_classes = ['Human & Animal', 'Interacting Materials']
mapping = {
    'dog': 0,
    'rooster': 0,
    'pig': 0,
    'cow': 0,
    'frog': 0,
    'cat': 0,
    'hen': 0,
    'insects': 0,
    'sheep': 0,
    'crow': 0,
    'rain': 1,
    'sea_waves': 1,
    'crackling_fire': 1,
    'crickets': 0,
    'chirping_birds': 0,
    'water_drops': 1,
    'wind': 1,
    'pouring_water': 1,
    'toilet_flush': 1,
    'thunderstorm': 1,
    'crying_baby': 0,
    'sneezing': 0,
    'clapping': 1,
    'breathing': 0,
    'coughing': 0,
    'footsteps': 1,
    'laughing': 0,
    'brushing_teeth': 1,
    'snoring': 0,
    'drinking_sipping': 1,
    'door_wood_knock': 1,
    'mouse_click': 1,
    'keyboard_typing': 1,
    'door_wood_creaks': 1,
    'can_opening': 1,
    'washing_machine': 1,
    'vacuum_cleaner': 1,
    'clock_alarm': 1,
    'clock_tick': 1,
    'glass_breaking':1,
    'helicopter': 1,
    'chainsaw': 1,
    'siren': 1,
    'car_horn': 1,
    'engine': 1,
    'train': 1,
    'church_bells': 1,
    'airplane': 1,
    'fireworks': 1,
    'hand_saw': 1,
}
dataset['h_target'] = None
for index, row in dataset.iterrows():
    target = row['target']
    classes[target] = row['category']
    dataset.loc[index, 'h_target'] = mapping[row['category']]

## Getting Preprocessed Data
We allow for previously preprocessed data to be retrieved for faster training turnaround. If the fold has been preprocessed, it is loaded but if not it is processed and saved.

In [5]:
df = ps.preprocess_fold(dataset, 
                        kind='mfcc', 
                        blocksize=blocksize, 
                        overlap=overlap
                       )

In [6]:
df.describe()

,mfcc_2_std,mfcc_2_mean,mfcc_2_noise,mfcc_3_std,mfcc_3_mean,mfcc_3_noise,mfcc_4_std,mfcc_4_mean,mfcc_4_noise,mfcc_5_std,...,sflat_mean,sflat_noise,sroll_std,sroll_mean,sroll_noise,rmse_std,rmse_mean,rmse_noise,h_target,target
count,30947.000000,30947.000000,30947.000000,30947.000000,30947.000000,30947.000000,30947.000000,30947.000000,30947.000000,30947.000000,...,3.094700e+04,3.094700e+04,3.094700e+04,30947.000000,30947.000000,30947.000000,30947.000000,30947.000000,30947.000000,30947.000000
mean,73.719667,46.264479,38.909376,33.446607,30.420424,30.067026,29.524960,29.485805,29.035557,28.434550,...,-1.535995e-02,-1.984839e-02,-1.938723e-02,14.260070,25.679845,5.237762,22.727680,4.911355,0.676673,24.927586
std,30.473942,18.374510,17.074185,14.922491,14.885459,15.237449,15.111426,15.600575,15.600374,15.252728,...,2.086369e-01,2.085811e-01,2.070859e-01,22.763022,35.058448,15.400834,17.316189,3.410737,0.467754,14.392002
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-7.723873e-01,-7.458212e-01,-7.421038e-01,3.736712,6.454405,1.436722,0.000000,0.510542,0.000000,0.000000
25%,54.270596,34.628251,27.134954,22.152604,17.779317,16.509955,16.142753,15.170072,14.981445,14.189170,...,-1.610912e-01,-1.659554e-01,-1.654091e-01,9.477854,17.833900,3.478282,14.893102,2.640077,0.000000,12.000000
50%,74.850493,46.716638,38.133677,33.581847,30.752174,30.586093,30.338455,30.395567,30.146264,29.760050,...,-4.367921e-09,-5.277997e-09,-8.137921e-09,11.984384,21.852503,4.507795,18.785621,4.292363,1.000000,25.000000
75%,93.894152,58.166543,50.129512,44.366403,41.731754,42.094875,41.359002,41.674235,40.984961,40.346104,...,1.205989e-01,1.163689e-01,1.146088e-01,15.742905,27.626522,5.601991,25.275695,6.459832,1.000000,38.000000
max,208.957500,123.849951,118.728586,92.085252,89.965195,89.935857,113.100970,118.224343,120.085079,113.077150,...,8.057657e-01,8.572066e-01,7.961318e-01,2206.362673,5031.091987,2444.047253,620.895529,156.366775,1.000000,49.000000


In [7]:
X = df.drop(['target','h_target'], axis=1)
y = df['target']
yy = df['h_target']

# Training

In [8]:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline

components = list(range(5,129))

clf = make_pipeline(
    MinMaxScaler(),
    SVC(max_iter=10000)
)

In [9]:
X.head()

,mfcc_2_std,mfcc_2_mean,mfcc_2_noise,mfcc_3_std,mfcc_3_mean,mfcc_3_noise,mfcc_4_std,mfcc_4_mean,mfcc_4_noise,mfcc_5_std,...,sband_noise,sflat_std,sflat_mean,sflat_noise,sroll_std,sroll_mean,sroll_noise,rmse_std,rmse_mean,rmse_noise
0,29.566979,38.581673,23.175071,24.097116,14.293689,13.274004,9.751989,10.407591,10.445341,14.507037,...,-0.249533,0.061410,-0.377001,0.093688,-0.244597,10.601514,20.214723,3.363856,18.003823,2.721278
1,36.087553,42.385458,41.323666,42.334032,40.613859,46.154355,39.037316,36.109261,37.146942,63.689873,...,-0.032375,-0.342203,-0.156344,0.210893,0.077557,19.903433,17.363853,5.242874,24.422219,6.690390
2,41.118476,57.628917,43.107704,33.185557,54.154237,41.865626,36.732927,36.665436,51.713698,44.632980,...,0.010137,-0.019897,-0.069090,0.309464,0.088745,21.462776,17.886832,6.170742,20.789183,7.557844
3,40.309076,62.498436,47.926910,36.613700,47.454525,44.457627,51.339283,34.410413,54.180220,48.870488,...,-0.135543,0.310175,-0.074617,-0.081635,-0.081316,21.383087,18.053033,6.724872,23.753132,8.544408
4,47.369925,81.112403,42.470077,51.519435,48.344907,56.496477,67.439424,52.695489,56.905152,60.244643,...,0.360566,0.104958,-0.019650,0.133368,0.006017,14.954617,18.298619,5.977200,28.201613,7.145342


In [10]:
scores = cross_validate(clf, 
                        X, yy, 
                        cv=10, 
                        scoring=['accuracy', 'f1', 'recall', 'precision', 'roc_auc'],
                        n_jobs=-1
                       )

In [11]:
default_stats = pd.DataFrame(scores)
default_stats.head()

,fit_time,score_time,test_accuracy,test_f1,test_precision,test_recall,test_roc_auc,train_accuracy,train_f1,train_precision,train_recall,train_roc_auc
0,153.168667,48.327644,0.676680,0.807166,0.676680,1.000000,0.684685,0.676672,0.807161,0.676672,1.000000,0.691679
1,157.727300,50.688501,0.675929,0.806632,0.676366,0.999045,0.626220,0.676504,0.807042,0.676626,0.999735,0.708445
2,156.395611,50.101707,0.676575,0.807092,0.676575,1.000000,0.651732,0.676684,0.807169,0.676684,1.000000,0.699905
3,153.352353,48.380455,0.676575,0.807092,0.676575,1.000000,0.662400,0.676684,0.807169,0.676684,1.000000,0.696184
4,156.609837,50.256912,0.676575,0.807092,0.676575,1.000000,0.665836,0.676684,0.807169,0.676684,1.000000,0.704070


In [12]:
default_stats.mean()

fit_time           154.922273
score_time          49.385974
test_accuracy        0.676576
test_f1              0.807093
test_precision       0.676642
test_recall          0.999857
test_roc_auc         0.673453
train_accuracy       0.676655
train_f1             0.807149
train_precision      0.676667
train_recall         0.999973
train_roc_auc        0.694709
dtype: float64

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('scaler', MinMaxScaler()),
    ('feat', PCA(n_components=None)),
    ('classify', SVC())
])

params = {
    'feat__n_components': range(2, 100)
}

gridsrc = GridSearchCV(clf, 
                        cv=10, 
                        param_grid=params,
                        scoring=['accuracy', 'f1', 'recall', 'precision', 'roc_auc'],
                        verbose=2,
                        n_jobs=5,
                        refit=False
                       )
gridsrc.fit(X,yy)

Fitting 10 folds for each of 98 candidates, totalling 980 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed: 10.8min
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed: 61.1min
[Parallel(n_jobs=5)]: Done 355 tasks      | elapsed: 178.9min
[Parallel(n_jobs=5)]: Done 638 tasks      | elapsed: 420.6min
[Parallel(n_jobs=5)]: Done 980 out of 980 | elapsed: 869.5min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('feat', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classify', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid='warn', n_jobs=5,
       param_grid={'feat__n_components': range(2, 100)},
       pre_dispatch='2*n_jobs', refit=False, return_train_score='warn',
       scoring=['accuracy', 'f1', 'recall', 'precision', 'roc_auc'],
       verbose=2)

In [14]:
pca_stats = pd.DataFrame(gridsrc.cv_results_)
pca_stats.head()

,mean_fit_time,mean_score_time,mean_test_accuracy,mean_test_f1,mean_test_precision,mean_test_recall,mean_test_roc_auc,mean_train_accuracy,mean_train_f1,mean_train_precision,...,std_test_accuracy,std_test_f1,std_test_precision,std_test_recall,std_test_roc_auc,std_train_accuracy,std_train_f1,std_train_precision,std_train_recall,std_train_roc_auc
0,18.888287,7.162975,0.676673,0.807162,0.676673,1.0,0.539616,0.676673,0.807162,0.676673,...,0.000103,0.000073,0.000103,0.0,0.034501,0.000011,0.000008,0.000011,0.0,0.026233
1,27.079152,7.285370,0.676673,0.807162,0.676673,1.0,0.581242,0.676673,0.807162,0.676673,...,0.000103,0.000073,0.000103,0.0,0.038606,0.000011,0.000008,0.000011,0.0,0.020248
2,26.310083,7.434330,0.676673,0.807162,0.676673,1.0,0.566401,0.676673,0.807162,0.676673,...,0.000103,0.000073,0.000103,0.0,0.019630,0.000011,0.000008,0.000011,0.0,0.023998
3,28.278228,7.600838,0.676673,0.807162,0.676673,1.0,0.569278,0.676673,0.807162,0.676673,...,0.000103,0.000073,0.000103,0.0,0.027687,0.000011,0.000008,0.000011,0.0,0.016723
4,32.810922,7.769433,0.676673,0.807162,0.676673,1.0,0.578046,0.676673,0.807162,0.676673,...,0.000103,0.000073,0.000103,0.0,0.021658,0.000011,0.000008,0.000011,0.0,0.012554


In [15]:
for col in pca_stats.columns:
    print(col)

mean_fit_time
mean_score_time
mean_test_accuracy
mean_test_f1
mean_test_precision
mean_test_recall
mean_test_roc_auc
mean_train_accuracy
mean_train_f1
mean_train_precision
mean_train_recall
mean_train_roc_auc
param_feat__n_components
params
rank_test_accuracy
rank_test_f1
rank_test_precision
rank_test_recall
rank_test_roc_auc
split0_test_accuracy
split0_test_f1
split0_test_precision
split0_test_recall
split0_test_roc_auc
split0_train_accuracy
split0_train_f1
split0_train_precision
split0_train_recall
split0_train_roc_auc
split1_test_accuracy
split1_test_f1
split1_test_precision
split1_test_recall
split1_test_roc_auc
split1_train_accuracy
split1_train_f1
split1_train_precision
split1_train_recall
split1_train_roc_auc
split2_test_accuracy
split2_test_f1
split2_test_precision
split2_test_recall
split2_test_roc_auc
split2_train_accuracy
split2_train_f1
split2_train_precision
split2_train_recall
split2_train_roc_auc
split3_test_accuracy
split3_test_f1
split3_test_precision
split3_test_recal

In [16]:
pca_stats_red = pca_stats[
    ['param_feat__n_components', 
     'mean_test_roc_auc', 
     'mean_test_accuracy', 
     'mean_test_recall', 
     'mean_test_precision']
]

pca_stats_red.sort_values(
             ['mean_test_roc_auc', 
              'mean_test_accuracy', 
              'mean_test_recall', 
              'mean_test_precision']
             , ascending=False)

,param_feat__n_components,mean_test_roc_auc,mean_test_accuracy,mean_test_recall,mean_test_precision
70,72,0.698478,0.676673,1.0,0.676673
77,79,0.698231,0.676673,1.0,0.676673
80,82,0.698171,0.676673,1.0,0.676673
79,81,0.698121,0.676673,1.0,0.676673
68,70,0.698076,0.676673,1.0,0.676673
69,71,0.698025,0.676673,1.0,0.676673
73,75,0.697909,0.676673,1.0,0.676673
74,76,0.697888,0.676673,1.0,0.676673
71,73,0.697884,0.676673,1.0,0.676673
76,78,0.697812,0.676673,1.0,0.676673


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('scaler', MinMaxScaler()),
    ('feat', KernelPCA(n_components=None)),
    ('classify', SVC())
])

params = {
    'feat__n_components': range(2, 100),
    'feat__kernel': ['poly', 'rbf', 'sigmoid', 'cosine']
}

gridsrc = GridSearchCV(clf, 
                        cv=10, 
                        param_grid=params,
                        scoring=['accuracy', 'f1', 'recall', 'precision', 'roc_auc'],
                        verbose=2,
                        n_jobs=5,
                        refit=False
                       )
gridsrc.fit(X,yy)

Fitting 10 folds for each of 392 candidates, totalling 3920 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed: 59.5min


KeyboardInterrupt: 

In [ ]:
kpca_stats = pd.concat(dfs, ignore_index=True)
kpca_stats.head()

In [ ]:
kpca_stats.groupby(['kernel','components']).mean().sort_values(['test_accuracy', 'test_recall', 'test_precision'], ascending=False)